In [ ]:
!pip install tensorflow-gpu==1.9.0 Keras==2.1.6

In [1]:
!sudo apt-get install graphviz -y


Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.38.0-17).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [2]:
!pip install pydot


distributed 1.21.8 requires msgpack, which is not installed.


In [3]:
from tensorflow.python import keras as tf_keras


ImportError: Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/ubuntu/anaconda3/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/ubuntu/anaconda3/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow ru

In [ ]:
from tf_keras import backend as K


In [ ]:
import pydot

import numpy as np
#import keras


from tensorflow.python import keras

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.metrics import categorical_crossentropy
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import Model


from tensorflow.python.keras.layers import Dense, Activation
from tensorflow.python.keras.optimizers import Adam, SGD, RMSprop



from sklearn.metrics import confusion_matrix



#from tensorflow.python.keras.applications.vgg16 import VGG16
#from tensorflow.python.keras import models
#from tensorflow.python.keras import layers

import itertools
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import tensorflow.python.keras.utils.plot_model

In [ ]:
from tensorflow.python.keras.utils import vis_utils

In [ ]:

#from tensorflow.python.keras.applications import imagenet_utils
from tensorflow.python.keras.utils.vis_utils import model_to_dot

In [ ]:
tensorflow.python.keras.layers

In [ ]:
from IPython.display import SVG


In [ ]:
!ls data

In [ ]:
train_folder = "data/train"
valid_folder = "data/valid"

In [ ]:

IMAGE_SIZE=224
BATCH_SIZE=64
NUM_CLASSES=120

In [ ]:
!ls data/train/ | head

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_generator = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input
                                     ,rotation_range=45
                                     ,width_shift_range=0.2
                                     ,height_shift_range=0.2
                                     ,shear_range=0.2
                                     ,zoom_range=0.25
                                     ,horizontal_flip=True
                                     ,fill_mode='nearest'
                                  )

train_batches = train_generator.flow_from_directory(
    train_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE,shuffle=True, seed=13,class_mode='categorical')



valid_generator =  ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)


valid_batches = valid_generator.flow_from_directory(
    valid_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE)

In [ ]:
NUM_CLASSES = train_batches.num_classes

In [ ]:
from tensorflow.python.keras.applications.mobilenet import MobileNet


In [ ]:
base_model = MobileNet()

In [ ]:
#base_model.summary()

In [ ]:
x = base_model.layers[-6].output
#predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
#!pip install tensorflow

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
#model.summary()

In [ ]:
#model.layers[:-1]

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
    

In [ ]:
#SVG(model_to_dot(model,show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))


In [ ]:
#model.summary()

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])

#model.compile(Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches, steps_per_epoch=train_batches.n//BATCH_SIZE, 
                    validation_data=valid_batches
                    ,  validation_steps=valid_batches.n // BATCH_SIZE, verbose=2,workers=4
                    , epochs=5
                   
                   )


In [ ]:
train_batches.class_indices.keys()

In [ ]:
train_batches.class_indices

In [ ]:
model.save('dog_breed_tf.h5')

In [ ]:
with open('labels.txt', 'w') as file_handler:
    for item in train_batches.class_indices.keys():
        file_handler.write("{}\n".format(item))

In [ ]:
#!pip install tensorflowjs 


In [ ]:
!mkdir model
!tensorflowjs_converter --input_format keras dog_breed_tf.h5 model/

In [ ]:
tra

In [ ]:
!gsutil cp /home/ubuntu/custom/deeplizrd/dog_breed.h5 gs://np-training-public/

In [ ]:
!pwd

In [ ]:

!ls /home/ubuntu/.kaggle/competitions/dog-breed-identification/train_folder/

